In [ ]:
import yaml
import add_packages
from pprint import pprint
import os
import pandas as pd
from tqdm.auto import tqdm

from toolkit.langchain import (
	document_loaders, text_splitters, text_embedding_models, stores, 
	prompts, utils, output_parsers, agents, documents, models,
	runnables, tools, chains
)

from toolkit import sql, utils

PATH_DATA = f"{add_packages.APP_PATH}/data/tdtu/FEEE"
FILE_CFG = "tdtu.yaml"
tqdm.pandas(desc="Processing")

with open(f"{add_packages.APP_PATH}/my_configs/{FILE_CFG}", 'r') as file:
	configs = yaml.safe_load(file)

In [ ]:
# llm = models.chat_openai
# embeddings = text_embedding_models.OpenAIEmbeddings()
llm = models.create_llm(provider="openai", version="gpt-4o-mini")
embeddings = text_embedding_models.OpenAIEmbeddings(model="text-embedding-3-large")

vectorstore = stores.faiss.FAISS

In [ ]:
my_sql_db = sql.MySQLDatabase()
# my_sql_db = sql.MySQLDatabase(
# 	dbname=os.getenv("SQL_DB_NEON"),
# 	host=os.getenv("SQL_HOST_NEON"),
# 	port=os.getenv("SQL_PORT_NEON"),
# 	user=os.getenv("SQL_USER_NEON"),
# 	password=os.getenv("SQL_PASSWORD_NEON"),
# )

# Data

## txt

### tdtu_feee_faq

In [ ]:
path_txt = f"{PATH_DATA}/tdtu_feee_faq.txt"

In [ ]:
loader_txt = document_loaders.TextLoader(path_txt)
doc_txt = loader_txt.load()

text_splitter = text_splitters.RecursiveCharacterTextSplitter(
	# chunk_size=500, chunk_overlap=100,
	separators=["##"], chunk_size=1000, chunk_overlap=0,
)
docs_txt = text_splitter.split_documents(doc_txt)
docs_txt = docs_txt[1:]

metadatas = {
	"data": "general information"
}
utils.remove_metadata(docs_txt, "source")
utils.update_metadata(docs_txt, metadatas)

In [ ]:
docs_txt_tdtu_feee_faq = docs_txt

### File 2

In [ ]:
path_txt = f"{PATH_DATA}/faq.txt"

In [ ]:
loader_txt = document_loaders.TextLoader(path_txt)
doc_txt = loader_txt.load()

text_splitter = text_splitters.RecursiveCharacterTextSplitter(
	# chunk_size=500, chunk_overlap=100,
	separators=["##"], chunk_size=150, chunk_overlap=0,
)
docs_txt = text_splitter.split_documents(doc_txt)
docs_txt = docs_txt[1:]

metadatas = {
	"data": "frequently asked questions"
}
utils.remove_metadata(docs_txt, "source")
utils.update_metadata(docs_txt, metadatas)

## csv

### NhanSu

In [ ]:
file_xlsx = "NhanSu.xlsx"
path_xlsx = f"{PATH_DATA}/{file_xlsx}"
path_xlsx_processed = f"{PATH_DATA}/{file_xlsx.split('.')[0]}1.xlsx"

In [ ]:
df = pd.read_excel(
	path_xlsx, 
 	# delimiter=";"
)

df.head()

# Prompting to get new col names

#### Process

In [ ]:
model = models.chat_openai

template1 = """\
...
{text}
"""

template2 = """\
...
{text}
"""

prompt_template1 = prompts.PromptTemplate.from_template(template1)
prompt_template2 = prompts.PromptTemplate.from_template(template2)

chain1 = prompt_template1 | model | output_parsers.StrOutputParser()
chain2 = prompt_template2 | model | output_parsers.StrOutputParser()

# chain = runnables.RunnablePassthrough.assign(
#   text=chain1
# ).assign(
#   text=chain2
# )


chain = runnables.RunnablePassthrough.assign(
  text=chain1
)

def process_xlsx_col(text: str) -> str:
  result = chain.invoke({"text": text})['text']
  return result

def capitalize_first_letter(s):
	return ' '.join([word.capitalize() for word in s.split()])

def change_col_value(df: pd.DataFrame, column_name: str, value, new_value):
	df[column_name] = df[column_name].replace(value, new_value)
	return df

def replace_col_value_if_contains(df, column_name, substring, new_substring):
	df[column_name] = df[column_name].str.replace(substring, new_substring)
	return df

# query = '...'
# result = process_xlsx_col(query)

# pprint(result)

In [ ]:
col_to_process = "Liên hệ"

df[col_to_process] = df[col_to_process].progress_apply(process_xlsx_col)


In [ ]:
# df.to_excel(f"{path_xlsx_processed}", index=False)

In [ ]:
path_xlsx = path_xlsx_processed

#### Load to sql

In [ ]:
my_table_schema = [
	"id SERIAL",
	"faculty TEXT",
	"name TEXT",
	"position TEXT",
	"major TEXT",
	"email TEXT",
	"office TEXT",
	"child_department TEXT",
	"PRIMARY KEY (id)",
]

my_table = sql.MySQLTable(
	name="tdtu_feee_personnel", 
	schema=my_table_schema,
	db=my_sql_db,
)
my_table.create()

db = stores.SQLDatabase.from_uri(my_sql_db.get_uri())

table_cols = [col_description.split(" ")[0] for col_description in my_table_schema][1:-1]


In [ ]:
# my_table.insert_from_dataframe(df)

In [ ]:
# df = pd.read_excel(path_xlsx)
# df.columns = table_cols

# my_table.insert_from_dataframe(df)

# cols = ['name', 'position', 'major', 'office', 'child_department']
# proper_nouns = [value for col in cols for value in my_table.get_discrete_values_col(col)]


In [ ]:
# all_proper_nouns.extend(proper_nouns)

In [ ]:
questions = ...
examples_questions_to_sql = ...

### ChuongTrinhDaoTao

In [ ]:
file_xlsx = "ChuongTrinhDaoTao.xlsx"
path_xlsx = f"{PATH_DATA}/{file_xlsx}"
path_xlsx_processed = f"{PATH_DATA}/{file_xlsx.split('.')[0]}1.xlsx"

In [ ]:
df = pd.read_excel(
	path_xlsx, 
 	# delimiter=";"
)

df.head()

# Prompting to get new col names

#### Process

In [ ]:
model = models.chat_openai

template1 = """\
...
{text}
"""

template2 = """\
...
{text}
"""

prompt_template1 = prompts.PromptTemplate.from_template(template1)
prompt_template2 = prompts.PromptTemplate.from_template(template2)

chain1 = prompt_template1 | model | output_parsers.StrOutputParser()
chain2 = prompt_template2 | model | output_parsers.StrOutputParser()

# chain = runnables.RunnablePassthrough.assign(
#   text=chain1
# ).assign(
#   text=chain2
# )


chain = runnables.RunnablePassthrough.assign(
  text=chain1
)

def process_xlsx_col(text: str) -> str:
  result = chain.invoke({"text": text})['text']
  return result

def capitalize_first_letter(s):
	return ' '.join([word.capitalize() for word in s.split()])

def change_col_value(df: pd.DataFrame, column_name: str, value, new_value):
	df[column_name] = df[column_name].replace(value, new_value)
	return df

def replace_col_value_if_contains(df, column_name, substring, new_substring):
	df[column_name] = df[column_name].str.replace(substring, new_substring)
	return df

# query = '...'
# result = process_xlsx_col(query)

# pprint(result)

In [ ]:
col_to_process = "Liên hệ"

df[col_to_process] = df[col_to_process].progress_apply(process_xlsx_col)


In [ ]:
df.to_excel(f"{path_xlsx_processed}", index=False)

In [ ]:
df

In [ ]:
path_xlsx = path_xlsx_processed

#### Load to sql

In [ ]:
my_table_schema = [
	"id SERIAL",
	"faculty TEXT",
	"study_field TEXT",
	"link TEXT",
	"program_type TEXT",
	"education_level TEXT",
	"introduction TEXT",
	"career_prospects TEXT",
	"outcome TEXT",
	"syllabub TEXT",
	"admission_candidates TEXT",
	"registration TEXT",
	"tuition TEXT",
 	"contact TEXT",
	"PRIMARY KEY (id)",
]
my_table = sql.MySQLTable(
	name="tdtu_feee_admission", 
	schema=my_table_schema,
	db=my_sql_db,
)
my_table.create()

table_cols = [col_description.split(" ")[0] for col_description in my_table_schema][1:-1]


In [ ]:
# my_table.insert_from_dataframe(df)

In [ ]:
# df = pd.read_excel(path_xlsx)
# df.columns = table_cols

# my_table.insert_from_dataframe(df)

# cols = ['faculty', 'study_field', 'program_type', 'education_level']
# proper_nouns = [value for col in cols for value in my_table.get_discrete_values_col(col)]


In [ ]:
# all_proper_nouns.extend(proper_nouns)

In [ ]:
questions = ...
examples_questions_to_sql = ...

# Vector store 

## txt

### tdtu_feee_faq

In [ ]:
qdrant_txt_tdtu_feee_faq = stores.QdrantStore(
  embeddings_provider="openai",
	embeddings_model="text-embedding-3-large",
	llm=models.chat_openai,
	search_type="mmr",
  configs=configs,
  distance="Cosine",
  retriever_types="base",
  **configs["vector_db"]["qdrant"]["tdtu_feee_faq"],
)

In [ ]:
# qdrant_txt_tdtu_feee_faq.add_documents(docs_txt_tdtu_feee_faq)

# Test

In [ ]:
my_chain_rag_tdtu_feee_faq = chains.MyRagChain(
	llm=llm,
	retriever=qdrant_txt_tdtu_feee_faq.retriever,
	is_debug=False,
	just_return_ctx=True,
	**configs["vector_db"]["qdrant"]["tdtu_feee_faq"],
)

tool_chain_rag_tdtu_feee_faq = my_chain_rag_tdtu_feee_faq.create_tool_chain_rag()


In [ ]:
examples_fewshot_tmp = dict(configs["sql"]["examples_questions_to_sql"]).values()
examples_questions_to_sql = [example for sublist in examples_fewshot_tmp for example in sublist]

proper_nouns = configs["sql"]["proper_nouns"]

my_sql_db = sql.MySQLDatabase()

cfg_sql = configs["sql"]
cfg_sql_tool = cfg_sql["tool"]

my_sql_chain = chains.MySqlChain(
	my_sql_db=my_sql_db,
	llm=llm,
	embeddings=embeddings,
	vectorstore=vectorstore,
	proper_nouns=proper_nouns,
	k_retriever_proper_nouns=4,
	examples_questions_to_sql=examples_questions_to_sql,
	k_few_shot_examples=5,
	sql_max_out_length=2000,
	is_sql_get_all=True,
	is_debug=False,
	tool_name=cfg_sql_tool["name"],
	tool_description=cfg_sql_tool["description"],
	tool_metadata=cfg_sql_tool["metadata"],
	tool_tags=cfg_sql_tool["tags"],
)

tool_chain_sql = my_sql_chain.create_tool_chain_sql()

In [ ]:
llm = models.create_llm(provider="openai", version="gpt-4o-mini")

tools = [
	tool_chain_rag_tdtu_feee_faq,
	tool_chain_sql,
]

system_message_custom = configs["prompts"]["system_message_tdtu"]
prompt = prompts.create_prompt_tool_calling_agent(system_message_custom)

agent = agents.MyStatelessAgent(
	llm=llm,
	tools=tools,
	prompt=prompt,
	agent_type="tool_calling",
	agent_verbose=False,
)

In [ ]:
res = []
async for chunk in agent.astream_events_basic(
	# "Người phụ trách bộ môn Điều khiển tự động khoa Điện",
	# "Các tiến sĩ trong khoa Điện", # adjust prompt to return all result
	# "Các thạc sĩ trong khoa Điện",
	# "Ký túc xá",
	"Chi tiết Chương trình Đại học về Tự động hoá khoa Điện",
  show_tool_call=True,
  history_type="mongodb",
  user_id=utils.generate_unique_id(thing="uuid_name"),
	session_id=utils.generate_unique_id(thing="uuid"),
):
	print(chunk, end="", flush=True)

	res.append(chunk)

In [ ]:
result = my_sql_chain.chain_sql.invoke({
	"question": 
   	"Các chương trình đào tạo khoa Điện có hình thức liên kết với các trường đại học nước ngoài?",
})

result

# Test

In [115]:

import add_packages
import boto3
import os
from operator import itemgetter
from loguru import logger
from typing import Union, Optional, List, Literal, AsyncGenerator, TypeAlias
from pydantic import BaseModel

from toolkit import utils

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, Runnable, RunnableParallel

from langchain.agents import (
	create_openai_tools_agent, create_openai_functions_agent, 
	create_react_agent, create_self_ask_with_search_agent,
	create_xml_agent, create_tool_calling_agent,
	AgentExecutor
)
from langchain_community.agent_toolkits import create_sql_agent

from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.prompts.chat import BaseChatPromptTemplate
from langchain_core.runnables import Runnable
from langchain_core.tools import BaseTool
from langchain_core.agents import (
	AgentActionMessageLog, AgentFinish, AgentAction
)
from langchain_core.messages import AIMessage, HumanMessage, ChatMessage

from langchain.agents.openai_assistant import OpenAIAssistantRunnable
from langchain.agents.format_scratchpad.openai_tools import (
	format_to_openai_tool_messages, 
)
from langchain.agents.format_scratchpad import (
	format_to_openai_function_messages,
)
from langchain.agents.output_parsers.openai_tools import (
	OpenAIToolsAgentOutputParser,
)

from langchain_openai import ChatOpenAI
from langchain_community.chat_message_histories.dynamodb import DynamoDBChatMessageHistory
from langchain_mongodb.chat_message_histories import MongoDBChatMessageHistory
#*==============================================================================

dynamodb = boto3.resource("dynamodb")

#*==============================================================================

TypeAgent: TypeAlias = Literal["tool_calling", "openai_tools", "react", "anthropic"]

TypeHistoryType: TypeAlias = Literal["in_memory", "dynamodb", "mongodb"]
TypeUserId: TypeAlias = Optional[str]

TypeSessionId: TypeAlias = Union[str, None]

#*==============================================================================

prompt_tpl_check_ans = """\
You are tasked with evaluating whether an AI's answer adequately addressed and satisfied the original question. Follow these steps carefully:

1. Review the following:

Original question:
<original_question>
{original_question}
</original_question>

AI's answer:
<ai_answer>
{ai_answer}
</ai_answer>

2. Analyze the AI's answer by considering the following:
   - Does the answer directly address the main points of the original question?
   - Is the information provided relevant and accurate?
   - Does the answer provide sufficient detail to satisfy the query?
   - Are there any parts of the original question left unanswered?
   - Ensure that all keywords in the answer correspond to the keywords in the question.

3. Based on your analysis, determine if the AI's answer adequately addressed and satisfied the original question.

4. Provide your response as follows:
   - If the answer adequately addressed and satisfied the query, output exactly: True
   - If the answer did not adequately address or satisfy the query, or if you cannot determine this due to lack of information or context, output exactly: ERROR

Do not provide any explanation or justification for your response. Your output must be either "True" or "ERROR" without any additional text.

Examples of correct outputs:
True
ERROR

Ensure your response is only one of these two options.
"""
prompt_check_ans = ChatPromptTemplate.from_template(prompt_tpl_check_ans)

def check_ans(original_question: str, ai_answer: str, llm=ChatOpenAI()):
	chain_check_ans = (
			{
				"original_question": itemgetter("original_question"),
				"ai_answer": itemgetter("ai_answer")
			}
			| prompt_check_ans
			| llm
			| StrOutputParser()
	).with_retry()

	result = chain_check_ans.invoke({
		"original_question": original_question,
		"ai_answer": ai_answer,
	})
	return result

#*------------------------------------------------------------------------------

prompt_tpl_res_if_not_satis = """\
You are tasked with generating a response to a user based on the number of retries for an AI-generated answer. You will be given three inputs: the AI's answer, the current retry count, and the maximum number of retries allowed.

Here are the inputs you will work with:

<ai_answer>
{ai_answer}
</ai_answer>

<current_retry>{current_retry}</current_retry>

<max_retry>{max_retry}</max_retry>

Follow these steps to generate the appropriate response:

1. Compare the value of current_retry to max_retry.

2. If current_retry is less or equals to than max_retry:
   - Inform the user that you will continue to retry to get the correct answer.
   
3. If current_retry is greater than max_retry:
   - Tell the user to please try again.

4. Ensure that your response is in the exact same language as the text in ai_answer. This means you should analyze the language used in ai_answer and formulate your response in that same language.

Remember, do not include any explanations or additional information. Your output should only be the appropriate message to the user, written in the same language as ai_answer.

Your response: \
"""
prompt_res_if_not_satis = ChatPromptTemplate.from_template(prompt_tpl_res_if_not_satis)


def response_if_not_satisfied(
  ai_answer: str, current_retry: int, max_retry: int,
  llm=ChatOpenAI()
):
	chain_res_if_not_satis = (
			{
				"ai_answer": itemgetter("ai_answer"),
				"current_retry": itemgetter("current_retry"),
				"max_retry": itemgetter("max_retry")
			}
			| prompt_res_if_not_satis
			| llm
			| StrOutputParser()
	).with_retry()
 
	result = chain_res_if_not_satis.invoke({
		"ai_answer": ai_answer,
		"current_retry": current_retry,
		"max_retry": max_retry,
	})
	return result

#*==============================================================================

class SchemaChatHistory(BaseModel):
	history_type: TypeHistoryType = "in_memory"
	user_id: TypeUserId = "admin"
	session_id: TypeSessionId = None
	history_size: Union[int, None] = 10

class ChatHistory:
	def __init__(self, schema: SchemaChatHistory):
		self.history_type = schema.history_type

		self.user_id = schema.user_id
		self.is_new_session = not bool(schema.session_id)
		self.session_id = schema.session_id if schema.session_id else utils.generate_unique_id("uuid_name")

		self.history_size = schema.history_size
	
		if self.history_type == "in_memory":
			self.chat_history = []
		elif self.history_type == "dynamodb":
			self.chat_history = DynamoDBChatMessageHistory(
				table_name="LangChainSessionTable", 
				session_id=self.session_id,
				key={
					"SessionId": self.session_id,
					"UserId": self.user_id,
				},
				history_size=self.history_size,
			)
		elif self.history_type == "mongodb":
			self.chat_history = MongoDBChatMessageHistory(
				session_id=self.session_id, # user name, email, chat id etc.
				connection_string=os.getenv("MONGODB_ATLAS_CLUSTER_URI"),
				database_name=os.getenv("MONGODB_DB_NAME"),
				collection_name=os.getenv("MONGODB_COLLECTION_NAME_MSG"),
			)			

		if self.is_new_session:
			welcome_msg = "Hello! How can I help you today?"
			if self.history_type == "in_memory":
				self.chat_history.append(AIMessage(welcome_msg))
			elif self.history_type == "dynamodb" or self.history_type == "mongodb":
				self.chat_history.add_ai_message(welcome_msg)

		if self.user_id: logger.info(f"User Id: {self.user_id}")
		logger.info(f"Session Id: {self.session_id}")
		logger.info(f"History Type: {self.history_type}")
	
	async def _add_messages_to_history(
		self,
		msg_user: str,
		msg_ai: str,
	):
		if self.history_type == "in_memory":
			if msg_user:
				self.chat_history.append(HumanMessage(msg_user))
			if msg_ai:
				self.chat_history.append(AIMessage(msg_ai))
		elif self.history_type == "dynamodb" or self.history_type == "mongodb":
			if msg_user:
				await self.chat_history.aadd_messages(messages=[HumanMessage(msg_user)])
			if msg_ai:
				await self.chat_history.aadd_messages(messages=[AIMessage(msg_ai)])

	async def _get_chat_history(self, is_truncate=True):
		if self.history_type == "in_memory":
			result = self.chat_history
			if is_truncate: result = result[-self.history_size:]
		elif self.history_type == "dynamodb":
			result = self.chat_history.messages
		elif self.history_type == "mongodb":
			result = await self.chat_history.aget_messages()
			if is_truncate: result = result[-self.history_size:]

		return result

	async def clear_chat_history(self):
		if self.history_type == "in_memory":
			self.chat_history = []
		elif self.history_type == "dynamodb" or self.history_type == "dynamodb":
			await self.chat_history.aclear()

class MyStatelessAgent:
	def __init__(
		self,
		llm: Union[BaseChatModel, None],
		tools: list[BaseTool],
		prompt: Union[BaseChatPromptTemplate, None],
	
		agent_type: Literal[
			"tool_calling", "openai_tools", "react", "anthropic"
		] = "tool_calling",
		agent_verbose: bool = False,
	):
		self.llm = llm
		self.my_tools = tools
		self.prompt = prompt

		self.agent_type = agent_type
		self.agent_verbose = agent_verbose
	
		self.agent = self._create_agent()
		self.agent_executor = AgentExecutor(
			agent=self.agent, tools=self.my_tools, verbose=self.agent_verbose,
			handle_parsing_errors=True,
			return_intermediate_steps=False,
		)

	def _create_agent(self) -> Runnable:
		logger.info(f"Agent type: {self.agent_type}")
	
		if self.agent_type == "tool_calling":
			return create_tool_calling_agent(self.llm, self.my_tools, self.prompt)
		elif self.agent_type == "openai_tools":
			return create_openai_tools_agent(self.llm, self.my_tools, self.prompt)
		elif self.agent_type == "react":
			return create_react_agent(llm=self.llm, tools=self.my_tools, prompt=self.prompt)
		elif self.agent_type == "anthropic": # todo
			return create_xml_agent(llm=self.llm, tools=self.my_tools, prompt=self.prompt)
		else:
			raise ValueError(
					"Invalid agent type. Supported types are 'openai_tools' and 'react'.")

	def _create_chat_history(
		self,
		history_type: TypeHistoryType = "mongodb",
		user_id: TypeUserId = "admin",
		session_id: TypeSessionId = None,
		history_size: Union[int, None] = 20,
	) -> ChatHistory:
	
		return ChatHistory(schema=SchemaChatHistory(
			history_type=history_type, user_id=user_id, session_id=session_id,
			history_size=history_size,
		)) 
	
	async def _add_messages_to_history(
		self,
		history: ChatHistory,
		history_type: TypeHistoryType,
		msg_user: str,
		msg_ai: str,
	):
		await history._add_messages_to_history(msg_user, msg_ai)
	
	async def invoke_agent(
		self,
		input_message: str,
		callbacks: Optional[List] = None,
		mode: Literal["sync", "async"] = "async",
	
		history_type: TypeHistoryType = "mongodb",
		user_id: TypeUserId = "admin",
		session_id: TypeSessionId = None,
	
		history_size: Union[int, None] = 20,
	):
		result = None

		history = self._create_chat_history(
			history_type, user_id, session_id, history_size,
		)

		input_data = {
			"input": input_message, "chat_history": await history._get_chat_history()
		}

		configs = {}
		configs["callbacks"] = callbacks if callbacks else []

		if mode == "sync":
			result = self.agent_executor.invoke(input_data, configs)
		elif mode == "async":
			result = await self.agent_executor.ainvoke(input_data, configs)

		result = result["output"]

		await self._add_messages_to_history(
			history=history,
			history_type=history_type,
			msg_user=input_message,
			msg_ai=result,
		)
	
		return result

	async def astream_events_basic_wrapper(
		self,
		input_message: str,
	):
		result = ""
		async for chunk in self.astream_events_basic(input_message):
			result += chunk
			print(chunk, end="", flush=True)
		return result

	async def astream_events_basic(
		self,
		input_message: str,

		history_type: TypeHistoryType = "mongodb",
		user_id: TypeUserId = utils.generate_unique_id(thing="uuid_name"),
		session_id: TypeSessionId = utils.generate_unique_id(thing="uuid"),
	
		show_tool_call: bool = False,
		history_size: Union[int, None] = 10,
	) -> AsyncGenerator[str, None]:
		"""
		async for chunk in agent.astream_events_basic("Hello"):
			print(chunk, end="", flush=True)
		"""

		history = self._create_chat_history(
			history_type, user_id, session_id, history_size,
		)

		result = ""
		is_result_satisfied = False
		max_retry = 2
		current_retry = 0

		""" used for debugging
		a = agent.events
		a = [x for x in agent.events if x["event"] == "on_chat_model_stream"]
		a_data = [x["data"] for x in a]
		a_data_chunk = [x["chunk"] for x in a_data]
		a_data_chunk_tool = [x for x in a if dict(a_data_chunk)["tool_call_chunks"]]
		a_metadata_sql_chain = [x for x in a if "..." in x["metadata"].keys()]
		"""
  
		# self.events = [] # debug

		while (not is_result_satisfied) and (current_retry <= max_retry):
			async for event in self.agent_executor.astream_events(
				input={"input": input_message, "chat_history": await history._get_chat_history()},
				version="v2",
			):
				# self.events.append(event) # debug
				event_event = event["event"]
				event_name = event["name"]
		
				try: event_data_chunk = event["data"]["chunk"]
				except: pass
				
				if event_event == "on_chat_model_stream":
					chunk = dict(event_data_chunk)["content"]
			
					if (event.get("metadata", {}).get("ls_stop") == ['\nSQLResult:']) \
							or ("is_my_sql_chain_run" in event.get("metadata", {})) \
							or ("is_my_rag_chain_run" in event.get("metadata", {})):
						continue

					result += chunk
					yield chunk
			
				if show_tool_call and event_event == "on_chain_stream":
					if event_name == "RunnableSequence":
						try:
							chunk: str = dict(event_data_chunk[0])["log"]
							chunk = f"`[TOOL - CALLING]` {chunk}"
				
							await self._add_messages_to_history(
								history=history,
								history_type=history_type,
								msg_user=None,
								msg_ai=chunk,
							)
							result += chunk
							yield chunk
						except:
							pass
				
					elif event_name == "RunnableLambda":
						try:
							chunk = dict(event_data_chunk[1])["content"]
							chunk = f"`[TOOL - RESULT]` {chunk}\n\n"
				
							await self._add_messages_to_history(
								history=history,
								history_type=history_type,
								msg_user=None,
								msg_ai=chunk,
							)
				
							result += chunk
							yield chunk
						except:
							pass
			
			current_retry += 1

			if check_ans(
     		original_question=input_message, ai_answer=result, llm=self.llm,
      ) != "ERROR":
				is_result_satisfied = True
				yield("\n")
			else:
		
				if current_retry <= max_retry:
					for _ in range(1):
						yield("\n")
			
					for res in response_if_not_satisfied(
						ai_answer=result,
						current_retry=current_retry,
						max_retry=max_retry,
						llm=self.llm,
					):
						yield res
					
					history = self._create_chat_history(
						history_type=history_type,
						user_id=utils.generate_unique_id(thing="uuid_name"), 
						session_id=utils.generate_unique_id(thing="uuid"), 
						history_size=history_size,
					)

				result = ""
    
		await self._add_messages_to_history(
			history=history,
			history_type=history_type,
			msg_user=input_message,
			msg_ai=result,
		)
	
def hello():
	...

In [116]:
llm = models.create_llm(provider="openai", version="gpt-3.5-turbo-0125")

tools = [
	tool_chain_rag_tdtu_feee_faq,
	tool_chain_sql,
]

system_message_custom = configs["prompts"]["system_message_tdtu"]
prompt = prompts.create_prompt_tool_calling_agent(system_message_custom)

agent = MyStatelessAgent(
	llm=llm,
	tools=tools,
	prompt=prompt,
	agent_type="tool_calling",
	agent_verbose=False,
)

2024-08-09 15:55:49.080 | INFO     | __main__:_create_agent:282 - Agent type: tool_calling


In [118]:
res = []
async for chunk in agent.astream_events_basic(
 
	"Giảng viên khoa quản trị kinh doanh",

  show_tool_call=False,
  history_type="mongodb",
  user_id=utils.generate_unique_id(thing="uuid_name"),
	session_id=utils.generate_unique_id(thing="uuid"),
):
	print(chunk, end="", flush=True)

	res.append(chunk)

2024-08-09 15:56:03.527 | INFO     | __main__:__init__:217 - User Id: Michael Friedman-03a9d67d-3cfc-4020-a78b-89099f980ac5
2024-08-09 15:56:03.527 | INFO     | __main__:__init__:218 - Session Id: d09d5451-00f2-4aee-9773-75731975b6c7
2024-08-09 15:56:03.528 | INFO     | __main__:__init__:219 - History Type: mongodb


Xin lỗi, hiện tại tôi không có thông tin về giảng viên của Khoa Quản trị kinh doanh tại Trường Đại học Tôn Đức Thắng. Bạn có thể thử lại sau hoặc liên hệ với bộ phận hỗ trợ của trường để biết thêm thông tin chi tiết.Bạn cần thông tin gì khác không ạ?
Xin lỗi, tôi sẽ tiếp tục thử để có được câu trả lời chính xác.

2024-08-09 15:56:17.023 | INFO     | __main__:__init__:217 - User Id: Grace Andrews-b86fe2ea-6180-4749-924b-7927135457bb
2024-08-09 15:56:17.025 | INFO     | __main__:__init__:218 - Session Id: cb72e054-c608-4355-ab0a-4e9ef306dfda
2024-08-09 15:56:17.025 | INFO     | __main__:__init__:219 - History Type: mongodb


Xin lỗi, hiện tại tôi không có thông tin về giảng viên của Khoa Quản trị kinh doanh tại Trường Đại học Tôn Đức Thắng. Bạn có thể thử lại sau hoặc liên hệ với bộ phận hỗ trợ của trường để biết thêm thông tin chi tiết.Bạn cần thông tin gì khác không ạ?
Xin lỗi, tôi sẽ tiếp tục thử để có được câu trả lời chính xác.

2024-08-09 15:56:30.474 | INFO     | __main__:__init__:217 - User Id: Sharon Torres-df0abb76-b218-4588-b21b-7b2e40162b90
2024-08-09 15:56:30.475 | INFO     | __main__:__init__:218 - Session Id: e5fe1154-d945-49b9-86e7-0197d0d0961c
2024-08-09 15:56:30.475 | INFO     | __main__:__init__:219 - History Type: mongodb


Xin lỗi, hiện tại tôi không có thông tin về giảng viên của Khoa Quản trị kinh doanh tại Trường Đại học Tôn Đức Thắng. Bạn có thể thử lại hoặc liên hệ với bộ phận hỗ trợ của trường để biết thêm thông tin chi tiết.Bạn cần thông tin gì khác không ạ?

In [ ]:
original_question = "Ai là người phụ trách khoa Điện - Điện tử?"
ai_answer = "Người phụ trách Khoa Điện - Điện tử tại Trường Đại học Tôn Đức Thắng là TS. Trần Thanh Phương, hiện đang giữ chức vụ Phó Trưởng Khoa - Phụ trách Khoa. Nếu bạn cần thêm thông tin về khoa hoặc các chương trình học, hãy cho tôi biết nhé!"
ai_answer = "Hihi you are my baby"

In [ ]:
for res in response_if_not_satisfied(
	ai_answer="Xin lỗi, hiện tại tôi không có thông tin về giảng viên của Khoa Quản trị kinh doanh tại Trường Đại học Tôn Đức Thắng. Bạn có thể thử lại sau hoặc liên hệ với bộ phận hỗ trợ của trường để biết thêm thông tin chi tiết.Bạn cần thông tin gì khác không ạ?",
	current_retry=1,
	max_retry=2,
):
  print(res, end="")